[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/satyaki-mallick/DeepLearningAssignment1/blob/master/Assignment_1.ipynb#scrollTo=7uut_aem5B5q)

# Assignment 1

<b>Group [fill in group number]</b>
* <b> Student 1 </b> : SATYAKI MALLICK + 1410881
* <b> Student 2 </b> : HUILIN + Huilin_student_no

**Reading material**
* [1] Mikolov, Tomas, et al. "[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)" Advances in neural information processing systems. 2013. 

<b><font color='red'>NOTE</font></b> When submitting your notebook, please make sure that the training history of your model is visible in the output. This means that you should **NOT** clean your output cells of the notebook. Make sure that your notebook runs without errors in linear order.



# Question 1 - Keras implementation (10 pt)

### Word embeddings
Build word embeddings with a Keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training. Use a window size of 2 to train the embeddings (`window_size` in the jupyter notebook). 

1. Build word embeddings of length 50, 150 and 300 using the Skipgram model
2. Build word embeddings of length 50, 150 and 300 using CBOW model
3. Analyze the different word embeddings:
    - Implement your own function to perform the analogy task (see [1] for concrete examples). Use the same distance metric as in the paper. Do not use existing libraries for this task such as Gensim. 
Your function should be able to answer whether an analogy like: "a king is to a queen as a man is to a woman" ($e_{king} - e_{queen} + e_{woman} \approx e_{man}$) is true. $e_{x}$ denotes the embedding of word $x$. We want to find the word $p$ in the vocabulary, where the embedding of $p$ ($e_p$) is the closest to the predicted embedding (i.e. result of the formula). Then, we can check if $p$ is the same word as the true word $t$.
    - Give at least 5 different  examples of analogies.
    - Compare the performance on the analogy tasks between the word embeddings and briefly discuss your results.

4. Discuss:
  - Given the same number of sentences as input, CBOW and Skipgram arrange the data into different number of training samples. Which one has more and why?


<b>HINT</b> See practical 3.1 for some helpful code to start this assignment.


### Import libraries

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import keras.backend as K
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence

# other helpful libraries
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
import pandas as pd

Using TensorFlow backend.


In [0]:
print(tf.__version__) #  check what version of TF is imported

2.2.0


### Import file

If you use Google Colab, you need to mount your Google Drive to the notebook when you want to use files that are located in your Google Drive. Paste the authorization code, from the new tab page that opens automatically when running the cell, in the cell below.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Navigate to the folder in which `alice.txt` is located. Make sure to start path with '/content/drive/My Drive/' if you want to load the file from your Google Drive.

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/'

/content/drive/My Drive/Colab Notebooks


In [0]:
file_name = 'alice.txt'
corpus = open(file_name).readlines()

### Data preprocessing

See Practical 3.1 for an explanation of the preprocessing steps done below.

In [0]:
# Removes sentences with fewer than 3 words
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

# remove punctuation in text and fit tokenizer on entire corpus
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)

# convert text to sequence of integer values
corpus = tokenizer.texts_to_sequences(corpus)
n_samples = sum(len(s) for s in corpus) # total number of words in the corpus
V = len(tokenizer.word_index) + 1 # total number of unique words in the corpus

In [0]:
n_samples, V

(27165, 2557)

In [0]:
# example of how word to integer mapping looks like in the tokenizer
print(list((tokenizer.word_index.items()))[:5])

[('the', 1), ('and', 2), ('to', 3), ('a', 4), ('it', 5)]


In [0]:
# parameters
window_size = 2
window_size_corpus = 4

In [0]:
# def save_embeddings_old(model, model_name):
#   weights = model.get_weights()
#   embedding = weights[0]
#   f = open(model_name + '.txt', 'w')
#   f.write(" ".join([str(V-1), str(dim)]))
#   f.write("\n")

#   for word,i in tokenizer.word_index.items():
#     f.write(word)
#     f.write(" ")
#     f.write(" ".join(map(str, list(embedding[i,:]))))
#     f.write("\n")
#   f.close()

In [0]:
def save_embeddings(model, model_name):
  weights = model.get_weights()
  embedding = weights[0]
  np.savetxt('new_' + model_name + '.txt',embedding)

## Task 1.1 - Skipgram
Build word embeddings of length 50, 150 and 300 using the Skipgram model.

In [0]:
#prepare data for skipgram
def generate_data_skipgram(corpus, window_size, V):
    # TODO Implement here
    # HINT: see Practical 3.1
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    # Add the input word
                    #in_words.append(word)
                    all_in.append(word)
                    # Add one-hot of the context words
                    all_out.append(to_categorical(words[i], V))
                                      
    return (np.array(all_in),np.array(all_out))

In [0]:
# create training data
x_skipgram , y_skipgram = generate_data_skipgram(corpus,window_size,V)

In [0]:
# x each word represented as a number and then a window created

In [0]:
# Sample of y:
# rows represent each word.
# columns represent total unique words
# sample array for y for input [1, 0, 3, 4, 5, 0, 2, 1]
# array([[ 0.,  1.,  0.,  0.,  0.,  0.],
#        [ 1.,  0.,  0.,  0.,  0.,  0.],
#        [ 0.,  0.,  0.,  1.,  0.,  0.],
#        [ 0.,  0.,  0.,  0.,  1.,  0.],
#        [ 0.,  0.,  0.,  0.,  0.,  1.],
#        [ 1.,  0.,  0.,  0.,  0.,  0.],
#        [ 0.,  0.,  1.,  0.,  0.,  0.],
#        [ 0.,  1.,  0.,  0.,  0.,  0.]])

In [0]:
x_skipgram.shape, y_skipgram.shape

((94556,), (94556, 2557))

In [0]:
V = y_skipgram.shape[1]

In [0]:
def skipgram_architechture(dim):
  model = Sequential()
  model.add(Embedding(input_dim=V, output_dim=dim, input_length=1, embeddings_initializer='glorot_uniform'))
  # not sure about the input length
  #model.add(Reshape((94556,), input_shape=(1,94556)))
  # above line or below line
  model.add(Reshape((dim,)))
  model.add(Dense(V, activation='softmax', kernel_initializer='glorot_uniform'))
  model.compile(optimizer='adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  return model

### Skipgram for Embedding Vector Length 50

In [0]:
dim = 50
skipgram50 = skipgram_architechture(dim)
skipgram50.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 50)             127850    
_________________________________________________________________
reshape_2 (Reshape)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2557)              130407    
Total params: 258,257
Trainable params: 258,257
Non-trainable params: 0
_________________________________________________________________


<b>HINT</b>: To increase training speed of your model, you can use the free available GPU power in Google Colab. Go to `Edit` --> `Notebook Settings` --> select `GPU` under `hardware accelerator`.

In [0]:
# train skipgram model
skipgram50.fit(x_skipgram, y_skipgram, batch_size=64, epochs=10)
save_embeddings(skipgram50, 'skipgram_50')

Epoch 1/10
1478/1478 [==============================] - 7s 5ms/step - loss: 7.8463 - accuracy: 4.0188e-04
Epoch 2/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8455 - accuracy: 9.4124e-04
Epoch 3/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8447 - accuracy: 0.0019
Epoch 4/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8438 - accuracy: 0.0045
Epoch 5/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8430 - accuracy: 0.0085
Epoch 6/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8422 - accuracy: 0.0134
Epoch 7/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8413 - accuracy: 0.0202
Epoch 8/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8405 - accuracy: 0.0259
Epoch 9/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8396 - accuracy: 0.0341
Epoch 10/10
1478/1478 [==============================] - 6s 4ms/step - los

### Skipgram for embedding vector length 150

In [0]:
# save embeddings for vectors of length 50, 150 and 300 using skipgram model
dim = 150
skipgram150 = skipgram_architechture(dim)
skipgram150.summary()
skipgram150.fit(x_skipgram, y_skipgram, batch_size=64, epochs=10)
save_embeddings(skipgram150, 'skipgram_150')


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1, 150)            383550    
_________________________________________________________________
reshape_5 (Reshape)          (None, 150)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2557)              386107    
Total params: 769,657
Trainable params: 769,657
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1478/1478 [==============================] - 10s 7ms/step - loss: 7.8462 - accuracy: 3.7015e-04
Epoch 2/10
1478/1478 [==============================] - 10s 7ms/step - loss: 7.8453 - accuracy: 5.1821e-04
Epoch 3/10
1478/1478 [==============================] - 9s 6ms/step - loss: 7.8445 - accuracy: 7.6145e-04
Epoch 4/10
1478/1478 [==============================] - 9s

### Skipgram for embedding vector length 300

In [0]:
dim = 300
skipgram300 = skipgram_architechture(dim)
skipgram300.fit(x_skipgram, y_skipgram, batch_size=32, epochs=10)
save_embeddings(skipgram300, 'skipgram_300')

Epoch 1/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8459 - accuracy: 3.9130e-04
Epoch 2/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8444 - accuracy: 6.6627e-04
Epoch 3/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8429 - accuracy: 0.0014
Epoch 4/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8415 - accuracy: 0.0022
Epoch 5/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8400 - accuracy: 0.0084
Epoch 6/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8385 - accuracy: 0.0127
Epoch 7/10
2955/2955 [==============================] - 21s 7ms/step - loss: 7.8371 - accuracy: 0.0164
Epoch 8/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8356 - accuracy: 0.0223
Epoch 9/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8341 - accuracy: 0.0257
Epoch 10/10
2955/2955 [==============================] - 20s 7ms/

## Task 1.2 - CBOW

Build word embeddings of length 50, 150 and 300 using CBOW model.

In [0]:
# prepare data for CBOW

# create training data

# create CBOW architecture

# train CBOW model

# save embeddings for vectors of length 50, 150 and 300 using CBOW model

Prepare data for CBOW



In [0]:
def generate_data_CBOW(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = [] #one real label wt
    for line in corpus:
        sentence_length = len(line)
      
        for index, word in enumerate(line):  #for each word in the line, we create a little neighborhood [left,right]
            left = index - window_size
            right = index + window_size + 1
      
            in_words = []   #neighbor words of wt, used as input to predict wt       
      
            for i in range(left, right):
                if 0 <= i < sentence_length and i != index:
                    # Add the input word
                    in_words.append(line[i])

            
            all_in.append(in_words)
            all_out.append(to_categorical(word,V))
      
    all_in = sequence.pad_sequences(all_in, maxlen=maxlen)
                                      
    return (np.array(all_in),np.array(all_out))

Create training data

In [0]:
x , y = generate_data_CBOW(corpus,window_size,V)

Create CBOW architecture

In [0]:
def cbow_architechture(dim):
    cbow = Sequential()
    cbow.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=window_size*2))
    cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
    cbow.add(Dense(V, kernel_initializer='glorot_uniform', activation='softmax'))
    #multiclass classification->categorical_crossentropy loss, optimizer->PPT02 p24
    cbow.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return cbow

###CBOW for Embedding Vector Length 50

Train CBOW model - embedding vector length 50

In [0]:
# dimension of word embedding
dim = 50

cbow = cbow_architechture(dim)

In [0]:
plot_model(cbow, show_shapes = True, show_layer_names=False)

In [0]:
cbow.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 4, 50)             127850    
_________________________________________________________________
lambda_3 (Lambda)            (None, 50)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 2557)              130407    
Total params: 258,257
Trainable params: 258,257
Non-trainable params: 0
_________________________________________________________________


In [0]:
# train skipgram model
cbow.fit(x, y, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
425/425 [==============================] - 2s 5ms/step - loss: 7.8466 - accuracy: 5.8899e-04
Epoch 2/10
425/425 [==============================] - 2s 5ms/step - loss: 7.8463 - accuracy: 7.3624e-04
Epoch 3/10
425/425 [==============================] - 2s 5ms/step - loss: 7.8461 - accuracy: 0.0010
Epoch 4/10
425/425 [==============================] - 2s 5ms/step - loss: 7.8458 - accuracy: 0.0014
Epoch 5/10
425/425 [==============================] - 2s 5ms/step - loss: 7.8456 - accuracy: 0.0019
Epoch 6/10
425/425 [==============================] - 2s 5ms/step - loss: 7.8454 - accuracy: 0.0025
Epoch 7/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8451 - accuracy: 0.0039
Epoch 8/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8449 - accuracy: 0.0053
Epoch 9/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8446 - accuracy: 0.0077
Epoch 10/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8444 - accuracy

Save embedding

In [0]:
save_embeddings(cbow, 'cbow_{dim}'.format(dim=dim))

###CBOW for Embedding Vector Length 150

Train CBOW model - embedding vector length 150

In [0]:
# dimension of word embedding
dim = 150

cbow = cbow_architechture(dim)

In [0]:
plot_model(cbow, show_shapes = True, show_layer_names=False)

In [0]:
cbow.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 4, 150)            383550    
_________________________________________________________________
lambda_1 (Lambda)            (None, 150)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2557)              386107    
Total params: 769,657
Trainable params: 769,657
Non-trainable params: 0
_________________________________________________________________


In [0]:
# train skipgram model
cbow.fit(x, y, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8466 - accuracy: 4.4174e-04
Epoch 2/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8463 - accuracy: 4.7856e-04
Epoch 3/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8460 - accuracy: 5.5218e-04
Epoch 4/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8458 - accuracy: 8.0987e-04
Epoch 5/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8455 - accuracy: 9.9393e-04
Epoch 6/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8453 - accuracy: 0.0015
Epoch 7/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8450 - accuracy: 0.0020
Epoch 8/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8448 - accuracy: 0.0024
Epoch 9/10
425/425 [==============================] - 3s 6ms/step - loss: 7.8445 - accuracy: 0.0034
Epoch 10/10
425/425 [==============================] - 3s 6ms/step - loss: 7.844

Save embedding

In [0]:
save_embeddings(cbow, 'cbow_{dim}'.format(dim=dim))

###CBOW for Embedding Vector Length 300

Train CBOW model - embedding vector length 300

In [0]:
# dimension of word embedding
dim = 300

cbow = cbow_architechture(dim)

In [0]:
plot_model(cbow, show_shapes = True, show_layer_names=False)

In [0]:
cbow.summary()

In [0]:
# train skipgram model
cbow.fit(x, y, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8462 - accuracy: 4.4174e-04
Epoch 2/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8459 - accuracy: 5.5218e-04
Epoch 3/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8456 - accuracy: 8.4668e-04
Epoch 4/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8453 - accuracy: 0.0012
Epoch 5/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8451 - accuracy: 0.0015
Epoch 6/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8448 - accuracy: 0.0018
Epoch 7/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8445 - accuracy: 0.0024
Epoch 8/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8442 - accuracy: 0.0034
Epoch 9/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8440 - accuracy: 0.0048
Epoch 10/10
425/425 [==============================] - 4s 9ms/step - loss: 7.8437 - accu

Save embedding

In [0]:
save_embeddings(cbow, 'cbow_{dim}'.format(dim=dim))

## Task 1.3 - Analogy function

Implement your own function to perform the analogy task (see [1] for concrete examples). Use the same distance metric as in [1]. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an analogy like: "a king is to a queen as a man is to a woman" ($e_{king} - e_{queen} + e_{woman} \approx e_{man}$) is true. 

In a perfect scenario, we would like that this analogy ( $e_{king} - e_{queen} + e_{woman}$) results in the embedding of the word "man". However, it does not always result in exactly the same word embedding. The result of the formula is called the expected or the predicted word embedding. In this context, "man" is called the true or the actual word $t$. We want to find the word $p$ in the vocabulary, where the embedding of $p$ ($e_p$) is the closest to the predicted embedding (i.e. result of the formula). Then, we can check if $p$ is the same word as the true word $t$.  

You have to answer an analogy function using each embedding for both CBOW and Skipgram model. This means that for each analogy we have 6 outputs. Show the true word (with distance similarity value between predicted embedding and true word embedding, i.e. `sim1`) , the predicted word (with distance similarity value between predicted embedding and the embedding of the word in the vocabulary that is closest to this predicted embedding, i.e. `sim2`) and a boolean answer whether the predicted word **exactly** equals the true word. 

<b>HINT</b>: to visualize the results of the analogy tasks , you can print them in a table. An example is given below.


| Analogy task | True word (sim1)  | Predicted word (sim2) | Embedding | Correct?|
|------|------|------|------|------|
|  queen is to king as woman is to ?	 | man (sim1) | predictd_word(sim2) | SG_50 | True / False|

* Give at least 5 different  examples of analogies.
* Compare the performance on the analogy s between the word embeddings and briefly discuss your results.

In [0]:
def embed(word, embedding, vocab_size=V, tokenizer=tokenizer):
  int_word = tokenizer.texts_to_sequences([word])[0]
  bin_word = to_categorical(int_word, V)
  return np.dot(bin_word, embedding)

In [0]:
def closest_word(predicted_embedding, word1_embedding, word2_embedding, word3_embedding, embedding):
  exclude_words = [predicted_embedding, word1_embedding, word2_embedding, word3_embedding]
  exclude_index = []
  for i in range(V):
    for j in exclude_words:
      if np.array_equal(j, embedding[[i]]):
        exclude_index.append(i)

  include_index = [x for x in range(V) if x not in exclude_index]

  least_dist = 100
  index = 0
  for i in include_index:  
    each_word = embedding[[i]]
    dist = cosine_similarity(predicted_embedding, each_word)   #sim1
    if dist < least_dist:
      least_dist = dist
      index = i
  
  mylist = list((tokenizer.word_index.items()))

  closest_word = [item[0] for item in mylist if item[1] == index]
  # print(least_dist)
  # print(least_dist[0][0])
  # print(closest_word)
  return least_dist[0][0], closest_word[0]

In [0]:
def analogy(word1, word2, word3, true_word):

  analogy = word2 + ' is to ' + word1 + ' as ' + word3 + ' is to?'
  df = pd.DataFrame(columns=['Analogy task', 'True word(sim1)', 'Predicted word(sim2)', 'Embedding', 'Correct?'])
  models = ['new_skipgram_50', 'new_skipgram_150', 'new_skipgram_300', 'new_cbow_50', 'new_cbow_150', 'new_cbow_300']

  dummy = []
  for i, model in enumerate(models):
    embedding = np.loadtxt(model + '.txt')
    
    word1_embedding = embed(word1, embedding)
    word2_embedding = embed(word2, embedding)
    word3_embedding = embed(word3, embedding)
    predicted_embedding = word1_embedding - word2_embedding + word3_embedding

    sim1 = cosine_similarity(predicted_embedding, embed(true_word, embedding))   #sim1 with true word

    pair = closest_word(predicted_embedding, word1_embedding, word2_embedding, word3_embedding, embedding)
    
    predicted_word = pair[1]
    sim2 = pair[0]
    
    
    sim1 = sim1[0][0]
    sim1 = '({val})'.format(val=sim1)
    t_word = true_word + sim1
    
    
    sim2 = '({val})'.format(val=sim2)
    p_word = predicted_word + sim2

    df.loc[i] = [analogy] + [t_word] + [p_word] + [model] + [true_word == predicted_word]

  return df

In [0]:
analogy('king', 'queen', 'woman', 'man')

,Analogy task,True word(sim1),Predicted word(sim2),Embedding,Correct?
0,queen is to king as woman is to?,man(0.2747060761868735),queen(-0.5592496528167128),new_skipgram_50,False
1,queen is to king as woman is to?,man(0.05237085339528883),queen(-0.5927744824086532),new_skipgram_150,False
2,queen is to king as woman is to?,man(0.15938018568449355),queen(-0.5980089172111602),new_skipgram_300,False
3,queen is to king as woman is to?,man(-0.055001982580041756),queen(-0.6710604283091975),new_cbow_50,False
4,queen is to king as woman is to?,man(0.031227230356360748),queen(-0.5427524069270854),new_cbow_150,False
5,queen is to king as woman is to?,man(-0.051832731540045016),queen(-0.6032603772060192),new_cbow_300,False


In [0]:
analogy('queen', 'king', 'man', 'woman')

In [0]:
# french to France as english to England
analogy('France', 'french', 'english', 'England')

,Analogy task,True word(sim1),Predicted word(sim2),Embedding,Correct?
0,french is to France as english is to?,England(0.08279263423405725),french(-0.6693843552112426),new_skipgram_50,False
1,french is to France as english is to?,England(0.009157961838430621),french(-0.5981663589861066),new_skipgram_150,False
2,french is to France as english is to?,England(-0.01861583430674554),french(-0.6088745577873178),new_skipgram_300,False
3,french is to France as english is to?,England(-0.04824368199079679),french(-0.5487317953851769),new_cbow_50,False
4,french is to France as english is to?,England(0.09603147874021359),french(-0.642151561116852),new_cbow_150,False
5,french is to France as english is to?,England(0.04251441444706375),french(-0.5914307670094049),new_cbow_300,False


In [0]:
# forget to forgetting as remember to remembering
analogy('forgetting', 'forget', 'remember', 'remembering')

,Analogy task,True word(sim1),Predicted word(sim2),Embedding,Correct?
0,forget is to forgetting as remember is to?,remembering(-0.09913164603413827),forget(-0.63004217355726),new_skipgram_50,False
1,forget is to forgetting as remember is to?,remembering(0.06814089575104072),forget(-0.5858552609108054),new_skipgram_150,False
2,forget is to forgetting as remember is to?,remembering(0.00846696534670178),forget(-0.5476451758806965),new_skipgram_300,False
3,forget is to forgetting as remember is to?,remembering(-0.08196106961549045),forget(-0.48945280895908233),new_cbow_50,False
4,forget is to forgetting as remember is to?,remembering(0.10967445606168344),forget(-0.6326756453634083),new_cbow_150,False
5,forget is to forgetting as remember is to?,remembering(0.010956263793287627),forget(-0.6004864519587253),new_cbow_300,False


In [0]:
# hand is to leg as to eye is to ear
analogy('knee', 'hand', 'eye', 'ear')

,Analogy task,True word(sim1),Predicted word(sim2),Embedding,Correct?
0,hand is to knee as eye is to?,ear(0.07856195492322865),hand(-0.6364739407157209),new_skipgram_50,False
1,hand is to knee as eye is to?,ear(0.10454801314273748),hand(-0.560377020895489),new_skipgram_150,False
2,hand is to knee as eye is to?,ear(0.13524304998920325),hand(-0.5646968284127875),new_skipgram_300,False
3,hand is to knee as eye is to?,ear(-0.1262367489371579),hand(-0.6293025006874384),new_cbow_50,False
4,hand is to knee as eye is to?,ear(-0.04453138474756916),hand(-0.4689737211157392),new_cbow_150,False
5,hand is to knee as eye is to?,ear(-0.02741340576249722),hand(-0.5776903747829307),new_cbow_300,False


In [0]:
# he is to his as to she is to hers
analogy('his', 'he', 'she', 'hers')

,Analogy task,True word(sim1),Predicted word(sim2),Embedding,Correct?
0,he is to his as she is to?,hers(0.09090712414425002),he(-0.6955926401938365),new_skipgram_50,False
1,he is to his as she is to?,hers(-0.046475233908444616),he(-0.5327758107042652),new_skipgram_150,False
2,he is to his as she is to?,hers(0.055003344127433546),he(-0.5959035526824187),new_skipgram_300,False
3,he is to his as she is to?,hers(0.05011060097141224),he(-0.6275309193846385),new_cbow_50,False
4,he is to his as she is to?,hers(-0.03526094842189002),he(-0.507630360235748),new_cbow_150,False
5,he is to his as she is to?,hers(-0.09557704132506899),he(-0.6285328148032823),new_cbow_300,False


In [0]:
analogy('he', 'his', 'hers', 'she')

## Task 1.4 - Discussion
Answer the following question:
* Given the same number of sentences as input, CBOW and Skipgram arrange the data into different number of training samples. Which one has more and why?


Answer:

Given a sentence of n unique words, and a window size of 2L for training. Skipgram will arrage 2L pairs of training samples for each word; therefore, Skipgram has 2nL training samples in total. On the other hand, CBOW uses the context words in the window differently. CBOW combines all 2L context words in the window into one traing sample; thus, CBOW has n training samples in total. The conclusion is that Skipgram has more training samples due to the way it pairs up each word with each of its context words in the window.

# Question 2 - Peer review (0 pt):
Finally, each group member must write a single paragraph outlining their opinion on the work distribution within the group. Did every group member
contribute equally? Did you split up tasks in a fair manner, or jointly worked through the exercises. Do you think that some members of your group deserve a different grade from others? You can use the table below to make an overview of how the tasks were divided:



| Student name | Task  |
|------|------|
|  student name 1  | task x |
| student name 2  | task x|
| everyone | task x|


In [0]:
# Does the order of first 2 words matter?
# How to find the predicted word from the predicted word embedding?
# How to find the closest word from the predicted word embedding?
# What is sim1 and sim2 ..?
# to determine closest, we should use mod of the value